<a href="https://colab.research.google.com/github/Ucchwas/ECG-Classification-using-CNN/blob/main/RNN_LSTM_Multiclass_version2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [20]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from keras.utils.np_utils import to_categorical
from sklearn.utils import class_weight
import warnings
warnings.filterwarnings('ignore')

In [21]:
df = pd.read_csv('/content/drive/MyDrive/Datasets/mitbih_train_Ihua.csv')

In [22]:
df.drop('1',axis='columns',inplace=True)
#print(df.dtypes)
df.fillna(0, inplace=True)
X = df.drop('Label1',axis='columns')
#Xs = df.drop('Label1',axis='columns')
#X = Xs[df['Label1'] != 1 & 2 & 3]
y = testLabels = df.Label1.astype(np.float32)
#ys = testLabels = df.Label1.astype(np.float32)
#y = ys[df['Label1'] != 1 & 2 & 3]

In [23]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.2,train_size=0.8, random_state=15, stratify=y)
#print(y_train.value_counts())
#print(y.value_counts())
#print(y.value_counts())
#print(len(X_train.columns))

#from tensorflow_addons import losses
import tensorflow as tf
from tensorflow import keras
from sklearn.metrics import confusion_matrix , classification_report

In [24]:
from keras.layers import Input, Convolution1D, BatchNormalization, MaxPooling1D, Flatten, Dense
from keras.models import Model

In [25]:
def LSTM_Model(X_train, y_train, X_test, y_test, loss, weights):
    # Convert DataFrame to NumPy arrays and reshape
    X_train = X_train.values.reshape(X_train.shape[0], 1, X_train.shape[1])
    X_test = X_test.values.reshape(X_test.shape[0], 1, X_test.shape[1])

    model = keras.Sequential([
        keras.layers.LSTM(256, input_shape=(1, X_train.shape[2]), activation='relu'),
        keras.layers.Dense(256, activation='relu'),
        keras.layers.Dense(5, activation='softmax'),
    ])

    # compile the model
    model.compile(optimizer='Adam', loss=loss, metrics=['accuracy'])

    if weights == -1:
        model.fit(X_train, y_train, epochs=2)
    else:
        model.fit(X_train, y_train, epochs=2, class_weight=weights)

    print(model.evaluate(X_test, y_test))

    y_preds = model.predict(X_train)
    y_preds = np.argmax(y_preds, axis=1)

    print("Classification Report: \n", classification_report(y_train, y_preds))
    return model, y_preds


In [26]:
model, y_preds = LSTM_Model(X_train, y_train, X_test, y_test, 'sparse_categorical_crossentropy', -1)
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_train, y_preds)
print("Cconfusion Matrix: \n",cm)

Epoch 1/2
2189/2189 [==============================] - 28s 12ms/step - loss: 0.2558 - accuracy: 0.9287
Epoch 2/2
548/548 [==============================] - 3s 5ms/step - loss: 0.1404 - accuracy: 0.9649
[0.14041250944137573, 0.9649363160133362]
2189/2189 [==============================] - 7s 3ms/step
Classification Report: 
               precision    recall  f1-score   support

         0.0       0.98      0.99      0.98     57978
         1.0       0.90      0.62      0.73      1778
         2.0       0.86      0.90      0.88      4630
         3.0       0.79      0.55      0.65       513
         4.0       0.97      0.96      0.96      5145

    accuracy                           0.97     70044
   macro avg       0.90      0.80      0.84     70044
weighted avg       0.97      0.97      0.96     70044

Cconfusion Matrix: 
 [[57192   126   518    44    98]
 [  612  1108    47     0    11]
 [  371     3  4175    31    50]
 [  165     0    67   281     0]
 [  186     0    40     0  4919]

In [27]:
model.save("RNN_LSTM_Multiclass.h5")

In [28]:
import tensorflow as tf

# Load the model
model = tf.keras.models.load_model("RNN_LSTM_Multiclass.h5")

In [31]:
# Convert to TensorFlow Lite model
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS, tf.lite.OpsSet.SELECT_TF_OPS]
converter._experimental_lower_tensor_list_ops = False
tflite_model = converter.convert()
open("RNN_LSTM_Multiclass.tflite", "wb").write(tflite_model)

2093704